In [8]:
import numpy as np
from scipy.optimize import linprog 

In [9]:
# X=np.array([[1,1,8],[1,4,5],[1,1,1],[1,4,4],[1,6,5],[1,6,4],[1,10,8],[1,8,1],[1,2,6],[1,2,3],[1,8,6],[1,8,3]])
# y=np.array([1,1,1,1,1,1,1,1,0,0,0,0])

In [10]:
def optimize(X,y,delta, alpha):
    (m,n) = X.shape
#     n = n-1
    K = (y == 0).sum()
    M = (y == 1).sum()
    
    A = np.zeros((2*n,m))
    B = np.zeros((m))
    for i in range(m):
        if y[i] == 0:     
            V = np.zeros((2,m))
            V[0][i] = 1
            V[1][i] = -1
            A = np.vstack((A,V))  
            for j in range(n):
                A[j][i] = X[i][j]
                A[j+n][i] = -1*X[i][j]
                B[i] = -delta
        else:  
            V = np.zeros((2,m))
            V[0][i] = -1
            V[1][i] = 1
            A = np.vstack((A,V)) 
            for j in range(n):
                A[j][i] = X[i][j]
                A[j+n][i] = -1*X[i][j]
                B[i] = delta
                
    A = np.vstack((A, np.zeros((n,m))))   # dimension of A is 2n+2m+n, m
    
    s1 = np.zeros((3*n+2*m))
    s2 = np.zeros((3*n+2*m))
    
    W = np.zeros((2*n+2*m+n))
    for i in range(m):
        if y[i] == 0:
            W[2*i+2*n] = 0                  #
            W[2*i+2*n+1] = 1/K              #
            s2[2*n+2*i] = -1
        else:
            W[2*i+2*n] = 0                  #
            W[2*i+2*n+1] = 1/M              #
            s1[2*n+2*i] = -1
            
    for i in range(n):
        W[i+2*n+2*m] = alpha   ##########
        
    A = A.T
    
    #Ax=B the equality contraints
    I = np.eye(n,n)
    C = np.zeros((2*n, 3*n+2*m))
    D = np.zeros((2*n+2))
    D[2*n+2-1] = -1
    D[2*n+2-2] = -1
    
    for i in range (n):
        C[i,i] = 1
        C[i+n,i] = -1
        C[i, i+n] = -1
        C[i+n, i+n] = 1
        C[i, i+2*n+2*m] = -1
        C[i+n, i+2*n+2*m] = -1
        
        # Cx<=D
    C = np.vstack((C, s1))
    C = np.vstack((C, s2))
    
    res = linprog(W, A_ub=C, b_ub=D, A_eq=A, b_eq=B,  bounds=(0, None), method = 'simplex') 
    # print('Optimal value:', res.fun, '\nX:', res.x)
    wd = res.x[:n]-res.x[n:2*n]
    si1 = np.zeros((m))
    si2 = np.zeros((m))
    for i in range(m):
        si1[i] = res.x[i*2+2*n]
    for i in range(m):
        si2[i] = res.x[i*2+2*n+1]
    
    # print(wd)
    # print(wd.shape)
    # print(X.shape)
    # print(y_hat)
    # wd is a row matrix
#     print(si1)
#     print(si2)
    return np.matrix(wd)


In [11]:
def y_hat(X, weights, delta, y):
    n = len(X)
#     X = X.T
#     print(weights.shape)
    
    y_res = np.asarray(np.dot(weights, X.T))
#     print(y_res)
#     print(type(y))
    for i in range(n):
#         print(y[0][i])
        if y_res[0][i] >= delta:
            y_res[0][i] = 1
        elif y_res[0][i] <= -delta:
            y_res[0][i] = 0
        else :
            y_res[0][i] = 1-y[i]
    return y_res[0]


In [12]:
X=np.array([[1,1,2],[1,1,5]])
y=np.array([0,1],dtype ='int32')
print(optimize(X,y,1, 0))

[[-4.66666667  0.          1.33333333]]


In [13]:
X = np.array([[1, 2, 3], [1, 8, 4],[1, 4, 5], [1, 4, 4],[1, 6, 5], [1, 6, 4],[1, 2, 6],[1, 8, 6]])
y = np.array([0,0,1,1,1,1,0,0])

# X = np.asarray(list_X,dtype='int32')
# y = np.asarray(list_y,dtype='int32')
# print(type(y))
delta = 1
wd = (optimize(X, y, delta, 0))
print(wd)


[[ 6.66666667  0.16666667 -1.5       ]]


In [14]:
y_out = y_hat(X, wd, delta, y)
print(y_out)
# print(y_out[5])

[1. 1. 0. 1. 0. 1. 0. 0.]


In [15]:
X=np.array([[1,1,8],[1,4,5],[1,1,1],[1,4,4],[1,6,5],[1,6,4],[1,10,8],[1,8,1],[1,2,6],[1,2,3],[1,8,6],[1,8,3]])
y=np.array([1,1,1,1,1,1,1,1,0,0,0,0])
delta = 1
wd = (optimize(X,y,delta, 0))
print(wd)

[[ 0.         -0.35294118  0.29411765]]


In [16]:
y_out = y_hat(X, wd, delta, y)
print(y_out)
# print(y_out[5])

[1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]


In [17]:
def classes(y,y_res):
    C1 = []
    C2 = []
    C3 = []
    C4 = []
    y = y.astype(int)
    y_res = y_res.astype(int)
    for i in range(len(y_res)): 
        if y[i] == 0 and y_res[i] == 0:
            C1.append(i)
        if y[i] == 1 and y_res[i] == 1:
            C2.append(i)
        if y[i] == 1 and y_res[i] == 0:
            C3.append(i)
        if y[i] == 0 and y_res[i] == 1:
            C4.append(i)
    return C1, C2, C3, C4

In [18]:
def find_Wa(X, C3, weights):
    res = 0
    y=0
    for i in C3:
        y = np.asarray(np.dot(weights, X[i].T))
        res = min(res, y[0][0])
#         print(res[0][0].shape)
    return -res+0.01

In [19]:
res = classes(y, y_out)
C3 = res[2]
find_Wa(X, C3, wd)



2.539411764705882

In [20]:
def find_Wb(X, C4, weights):
    res = 0
    y=0
    for i in C4:
        y = np.asarray(np.dot(weights, X[i].T))
        res = max(res, y[0][0])
#         print(res[0][0].shape)
    return -res-0.01

In [21]:

C4 = res[3]
find_Wb(X, C4, wd)


-1.0688235294117645

In [22]:
class Neuron:
    
    def __init__(self,n):
        self.X = np.zeros((1,n))
        self.inp = n
        self.A = None
        self.B = None
        self.weight = 0
        
    def insert(self,neuron_type, weight):     #### Type is a restricted variable, Type was always set to A, weigth to 1
        if neuron_type == 'A':
            self.A = Neuron(self.inp)
#             print(self.X)
#             print( np.array([weight]) )
            self.X = np.hstack((self.X, np.array([[weight]])))
        else:
            self.B = Neuron(self.inp)
            self.X = np.hstack((self.X, np.array([[weight]])))
    
#     def calculate(self,X):
#         if self.A != None and self.B != None:
#             y_1 = calculate(self.A)
#             y_2 = calculate(self.B)
#             return np.sum(np.multiply(self.X, np.vstack((X, y_1, y_2))))
#         elif self.A != None:
#             y_1 = calculate(self.A)
#             return np.sum(np.multiply(self.X, np.vstack((X, y_1))))
#         elif self.B != None:
#             y_2 = calculate(self.B)
#             return np.sum(np.multiply(self.X, np.vstack((X, y_2))))
#         else:
#             return np.sum(np.multiply(self.X, X))

In [23]:
def addneuron_A(X, y, W, C1, C3, C4, delta, alpha, curr_Neuron):
    n = X.shape[1]
    print('A')
    m = len(C1) + len(C3) + len(C4)
#     print(m)
    print((C1))
    print((C3))
    print((C4))
    y_A = np.zeros((m))
    X_A = np.zeros((m, n))
    for i in range(len(C1)):
        X_A[i] = X[C1[i]]
        y_A[i] = 0

    for i in range(len(C3)):
        X_A[i + len(C1)] = X[C3[i]]
        y_A[i + len(C1)] = 1

    for i in range(len(C4)):
        X_A[i + len(C1)+ len(C3)] = X[C4[i]]
        y_A[i + len(C1)+ len(C3)] = 0

    print(y_A)
    Wa = find_Wa(X, C3, W)
#         print(Wa)       
    W_A = optimize(X_A, y_A, delta, alpha)
    curr_Neuron.insert('A', Wa)
    curr_Neuron.A.X = W_A
    
  
    return X_A, y_A, W_A




In [40]:
def addneuron_B(X, y, W, C2, C3, C4, delta, alpha, curr_Neuron):
    n = X.shape[1]
    print('B')
    m = len(C2) + len(C3) + len(C4)
    print(m)
    y_B = np.zeros((m))
    X_B = np.zeros((m, n))
#         y_B = []
    print((C2))
    print((C3))
    print((C4))
    for i in range(len(C2)):
        X_B[i] = X[C2[i]]
        y_B[i] = 0

    for i in range(len(C3)):
        X_B[i + len(C2)] = X[C3[i]]
        y_B[i + len(C2)] = 0

    for i in range(len(C4)):
        X_B[i + len(C2)+ len(C3)] = X[C4[i]]
        y_B[i + len(C2)+ len(C3)] = 1


#     print(X_B)
    print(y_B)
    Wb = find_Wb(X, C4, W)
    W_B = optimize(X_B, y_B, delta, alpha)
    curr_Neuron.insert('B', Wb)
    curr_Neuron.B.X = W_B

    return X_B, y_B, W_B


In [43]:
def classifier(X, y, W, delta, alpha, curr_Neuron):
    print(y)
    y_res = y_hat(X, W, delta, y)
    print(y_res)
    C1, C2, C3, C4 = classes(y, y_res)
    print(len(C3))
     
    if len(C3)!= 0:
        X_A, y_A, W_A = addneuron_A(X, y, W, C1, C3, C4, delta, alpha, curr_Neuron)
        classifier(X_A, y_A, W_A, delta, alpha, curr_Neuron.A)
        
    if len(C4)!= 0:
        X_B, y_B, W_B = addneuron_B(X, y, W, C2, C3, C4, delta, alpha, curr_Neuron)
        classifier(X_B, y_B, W_B, delta, alpha, curr_Neuron.B)
        
    else:
        return 'Terminated'

In [44]:
X = np.array([[1,1,8],[1,4,5],[1,1,1],[1,4,4],[1,6,5],[1,6,4],[1,10,8],[1,8,1],[1,2,6],[1,2,3],[1,8,6],[1,8,3]])
y=np.array([1,1,1,1,1,1,1,1,0,0,0,0])
alpha = 0
delta = 1
curr_Neuron = Neuron(3)
W = optimize(X, y, delta, alpha)
curr_Neuron.X = W


classifier(X, y, W, delta, alpha, curr_Neuron)
# print(curr_Neuron.calculate(X))

[1 1 1 1 1 1 1 1 0 0 0 0]
[1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
7
A
[10, 11]
[1, 2, 3, 4, 5, 6, 7]
[8, 9]
[0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
[0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
[1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
6
A
[9]
[2, 3, 4, 5, 6, 7]
[0, 1, 10]
[0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
[0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
4
A
[0]
[1, 2, 3, 4]
[7, 8, 9]
[0. 1. 1. 1. 1. 0. 0. 0.]
[0. 1. 1. 1. 1. 0. 0. 0.]
[1. 0. 1. 0. 0. 0. 1. 1.]
3
A
[5]
[1, 3, 4]
[0, 6, 7]
[0. 1. 1. 1. 0. 0. 0.]
[0. 1. 1. 1. 0. 0. 0.]
[1. 1. 1. 1. 1. 0. 1.]
0
B
6
[1, 2, 3]
[]
[0, 4, 6]
[0. 0. 0. 1. 1. 1.]
[0. 0. 0. 1. 1. 1.]
[1. 1. 0. 0. 1. 0.]
2
A
[2]
[3, 5]
[0, 1]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[1. 1. 0. 0. 1.]
1
A
[3]
[2]
[0, 4]
[0. 1. 0. 0.]
[0. 1. 0. 0.]
[1. 1. 0. 0.]
0
B
2
[1]
[]
[0]
[0. 1.]
[0. 1.]
[0. 1.]
0
B
4
[1]
[2]
[0, 4]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 1.]
0
B
4
[2, 3]
[]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 0.]
1
A
[]
[3]
[0, 1]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
0
B
4


[1. 0. 0.]
[1. 0. 0.]
0
B
4
[2]
[3]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 1.]
0
B
4
[2, 3]
[]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 0.]
1
A
[]
[3]
[0, 1]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
0
B
4
[2]
[3]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 1.]
0
B
4
[2, 3]
[]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 0.]
1
A
[]
[3]
[0, 1]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
0
B
4
[2]
[3]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 1.]
0
B
4
[2, 3]
[]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 0.]
1
A
[]
[3]
[0, 1]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
0
B
4
[2]
[3]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 1.]
0
B
4
[2, 3]
[]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 0.]
1
A
[]
[3]
[0, 1]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
0
B
4
[2]
[3]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 1.]
0
B
4
[2, 3]
[]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 0.]
1
A
[]
[3]
[0, 1]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
0
B
4
[2]
[3]
[0, 1]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 1. 1.]
0
B
4
[2, 3]
[]
[0, 

KeyboardInterrupt: 